# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [4]:
# to do: homework :)
import dlt
import duckdb

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

Answer the following questions:

Question 1: What is the sum of the outputs of the generator for limit = 5?
Question 2: What is the 13th number yielded
I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
# Question 1: What is the sum of the outputs of the generator for limit = 5?
# Answer:

def square_root_generator(limit):
    n = 1
    while n <= limit:
      yield n ** 0.5
      n += 1

sum = 0
for sqrt in square_root_generator(5):
    sum += sqrt
print(sum)

8.382332347441762


In [6]:
# Question 2: What is the 13th number yielded
# Answer:

def square_root_generator(limit):
    n = 1
    while n <= limit:
      yield n ** 0.5
      n += 1

sqrt_list = [sqrt_value for sqrt_value in square_root_generator(13)]
print(sqrt_list[-1])

3.605551275463989


# 2. Append a generator to a table with existing data

Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
Append the second generator to the same table as the first.
After correctly appending the data, calculate the sum of all ages of people.

In [7]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_dataset')

info = pipeline.run(
    people_1(),
    table_name = "people_details",
    write_disposition = "replace",
    primary_key="ID"
)

print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables:')
display(conn.sql("show tables"))


print("\n\n\n Adding people_1() data")
people_details = conn.sql("SELECT * FROM people_details").df()
display(people_details)

info = pipeline.run(
    people_2(),
    table_name = "people_details",
    write_disposition = "append",
    primary_key="ID"
)

print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables:')
display(conn.sql("show tables"))


print("\n\n\n After adding people_2() data")
people_details = conn.sql("SELECT * FROM people_details").df()
display(people_details)

print("\n\n\n Sum of ages of all people")
people_details = conn.sql("SELECT SUM(age) as sum_of_ages FROM people_details").df()
display(people_details)

Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711253764.0380652 is LOADED and contains no failed jobs
Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_details      │
└─────────────────────┘




 Adding people_1() data


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1711253764.0380652,228WAoIMQNWtrg,None
1,2,Person_2,27,City_A,1711253764.0380652,K0c/VO8ZM7ekNw,None
2,3,Person_3,28,City_A,1711253764.0380652,dwTE43MfkwAh0Q,None
3,4,Person_4,29,City_A,1711253764.0380652,q/TIYfnUBAIWXw,None
4,5,Person_5,30,City_A,1711253764.0380652,9D5vQskCWOGRCA,None


Pipeline dlt_colab_kernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711253765.014142 is LOADED and contains no failed jobs
Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_details      │
└─────────────────────┘




 After adding people_2() data


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1711253764.0380652,228WAoIMQNWtrg,None
1,2,Person_2,27,City_A,1711253764.0380652,K0c/VO8ZM7ekNw,None
2,3,Person_3,28,City_A,1711253764.0380652,dwTE43MfkwAh0Q,None
3,4,Person_4,29,City_A,1711253764.0380652,q/TIYfnUBAIWXw,None
4,5,Person_5,30,City_A,1711253764.0380652,9D5vQskCWOGRCA,None
5,3,Person_3,33,City_B,1711253765.014142,fNp1n/b61U0gPw,Job_3
6,4,Person_4,34,City_B,1711253765.014142,7nCV0kUleilvhg,Job_4
7,5,Person_5,35,City_B,1711253765.014142,mlOeB4ngaCF9Ag,Job_5
8,6,Person_6,36,City_B,1711253765.014142,M49+FU5IYTuaFA,Job_6
9,7,Person_7,37,City_B,1711253765.014142,nC4UlofpzjoyPw,Job_7





 Sum of ages of all people


,sum_of_ages
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: Calculate the sum of ages of all the people loaded as described above.

In [8]:
# Answer:

import dlt
import duckdb

# Adding people_1 with merge

pipeline_merge = dlt.pipeline(destination='duckdb', dataset_name='people_dataset_merge')

info = pipeline_merge.run(
    people_1(),
    table_name = "people_details",
    write_disposition = "replace",
    primary_key="ID"
)

print(info)

conn = duckdb.connect(f"{pipeline_merge.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline_merge.dataset_name}'")
print('Loaded tables:')
display(conn.sql("show tables"))


print("\n\n\n Adding people_1() data")
people_details = conn.sql("SELECT * FROM people_details").df()
display(people_details)


# Adding people_2 with merge

info = pipeline_merge.run(
    people_2(),
    table_name = "people_details",
    write_disposition = "merge",
    primary_key="ID"
)

print(info)

conn = duckdb.connect(f"{pipeline_merge.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline_merge.dataset_name}'")
print('Loaded tables:')
display(conn.sql("show tables"))


print("\n\n\n After adding people_2() data")
people_details = conn.sql("SELECT * FROM people_details").df()
display(people_details)

# Calculate sum of ages after merge

print("\n\n\n Sum of ages of all people with merge")
people_details = conn.sql("SELECT SUM(age) as sum_of_ages FROM people_details").df()
display(people_details)

Pipeline dlt_colab_kernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset_merge
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711253765.9012039 is LOADED and contains no failed jobs
Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_details      │
└─────────────────────┘




 Adding people_1() data


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1711253765.9012039,14fDdVfyCxcfVw,None
1,2,Person_2,27,City_A,1711253765.9012039,gn3+elalHrRMzg,None
2,3,Person_3,28,City_A,1711253765.9012039,FhV4gIFALIAs2g,None
3,4,Person_4,29,City_A,1711253765.9012039,evS9RQHCbSwI0A,None
4,5,Person_5,30,City_A,1711253765.9012039,SGNekcE1/mhSig,None


Pipeline dlt_colab_kernel_launcher load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset_merge
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711253766.7150629 is LOADED and contains no failed jobs
Loaded tables:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_details      │
└─────────────────────┘




 After adding people_2() data


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1711253765.9012039,14fDdVfyCxcfVw,None
1,2,Person_2,27,City_A,1711253765.9012039,gn3+elalHrRMzg,None
2,5,Person_5,35,City_B,1711253766.7150629,dTQEVZl+DmKViQ,Job_5
3,7,Person_7,37,City_B,1711253766.7150629,BXFxtXcrWk4roQ,Job_7
4,8,Person_8,38,City_B,1711253766.7150629,zprNbUnQi+BcLg,Job_8
5,4,Person_4,34,City_B,1711253766.7150629,iW8D8zPjTEzNAw,Job_4
6,3,Person_3,33,City_B,1711253766.7150629,J3XkGyM4u218eQ,Job_3
7,6,Person_6,36,City_B,1711253766.7150629,IyDEgGR6paiotg,Job_6





 Sum of ages of all people with merge


,sum_of_ages
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX